In [ ]:
import os

import numpy as np
import torch

import lib.dataset.dataset2D as dataUtils
from lib.dataset.dataset2D import VortexDataset2D
from lib.vortex.modules.efficienttrack.efficienttrack import EfficientTrack

from lib.config.project_manager import ProjectManager

project = ProjectManager()
project.load('Ralph_Test3D')

#project.create_new(
#    name = 'Ralph_Test3D',
#    dataset2D_path = '/home/trackingsetup/Documents/Ralph_Test2',
#    dataset3D_path = '/home/trackingsetup/Documents/Ralph_Test2')

cfg = project.get_cfg()

In [ ]:
training_set = VortexDataset2D(project.cfg, set='train', mode = 'keypoints')
val_set = VortexDataset2D(project.cfg, set='val',mode = 'keypoints')

In [ ]:
weights = '/home/trackingsetup/Downloads/EfficientTrack-d2_100.pth'

efficientTrack = EfficientTrack('train', cfg, weights)

In [ ]:
efficientTrack.train(training_set, val_set, 1000)

In [ ]:
import lib.vortex.modules.efficienttrack.darkpose as darkpose
import cv2
import matplotlib.pyplot as plt

weights = "/home/trackingsetup/Documents/Vortex/projects/Ralph_Test3D/models/efficienttrack/Run_20210827-173925/EfficientTrack-d2_40.pth"

efficientTrack = EfficientTrack('inference', cfg, weights)

In [ ]:

item = val_set.__getitem__(j)
j += 1
imgs = item[0]
img = ((imgs*cfg.DATASET.STD)+cfg.DATASET.MEAN)*255

imgs = torch.from_numpy(imgs).permute(2, 0, 1).float()

imgs = imgs.reshape(1,3,256,256)
imgs = imgs.cuda()
outputs = efficientTrack.model(imgs)
preds, maxvals = darkpose.get_final_preds(outputs[1].clamp(0,255).detach().cpu().numpy(), None)

print (maxvals)

colors = [(255,0,0), (255,0,0),(255,0,0),(255,0,0),(0,255,0),(0,255,0),(0,255,0),(0,255,0),(0,0,255),(0,0,255),(0,0,255),(0,0,255),(255,255,0),(255,255,0),(255,255,0), (255,255,0),
                  (0,255,255),(0,255,255),(0,255,255),(0,255,255), (255,0,255),(100,0,100),(100,0,100)]
for i,point in enumerate(preds[0]):
    if (maxvals[0][i]) > 100:
        cv2.circle(img, (int(point[0]*2), int(point[1])*2), 2, colors[i], thickness=5)
    else:
        cv2.circle(img, (int(point[0]*2), int(point[1])*2), 2, (100,100,100), thickness=5)

plt.imshow(img/255.)
#hetamap = cv2.resize(outputs[1].clamp(0,255).detach().cpu().numpy()[0][12]/255., (256,256), interpolation=cv2.cv2.INTER_NEAREST)
#plt.imshow(hetamap, alpha=0.6)
plt.imsave( 'test.png',img/255.)
#plt.savefig('test.png')